In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import random

from models.resnet import *
from wrapper import TestWrapper, ModelWrapper
import matplotlib.pyplot as plt
from advertorch.attacks import CarliniWagnerL2Attack, L2PGDAttack, L1PGDAttack, LinfPGDAttack, SparseL1DescentAttack
from attack import PGDAttack


In [2]:
transform_test = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True, num_workers=2)

In [3]:
class Evaluation:
    def __init__(self, model):
        self.correct = 0
        self.total = 0
        self.model = model
        self._nll = 0

    def evaluate(self, inputs, targets):
        with torch.no_grad():
            outputs = self.model(inputs)
            likelihood, predicted = outputs.max(1)
            self._nll += -torch.log(likelihood).sum().item()

            self.total += targets.size(0)
            self.correct += predicted.eq(targets).sum().item()
        
    @property
    def dr(self):
        return 1. * self.correct / self.total
    

In [6]:
from models.resnet import *
from models.dnn import net
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
# MIMO setting
ensembles = 1
# net = ResNet18(ensembles,10*ensembles).to(device)
net = net(ensembles).to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('checkpoint/ckpt_msd_mnist_maxepoch15.pth')
net.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']
optimizer = checkpoint['optimizer']

criterion = nn.CrossEntropyLoss()

net = TestWrapper(net, ensembles)
# optimizer = optim.Adam(net.parameters(), lr=0.01)
# attack = CarliniWagnerL2Attack(net, num_classes=10)
# attack = PGDAttack(net, alpha=[0.125], epsilon=[0.5], norm=[2], max_iteration=40, random_start=True)
# attack = PGDAttack(net, alpha=[2/255], epsilon=[8/255], norm=[float('inf')], max_iteration=40, random_start=True)
# attack = PGDAttack(net, alpha=[3.], epsilon=[12.], norm=[1], max_iteration=40, random_start=True)
attack = LinfPGDAttack(net, eps=0.3, nb_iter=100, eps_iter=0.01, rand_init=True)
# attack = L2PGDAttack(net, eps=2., nb_iter=200, eps_iter=0.1, rand_init=True)
# attack = SparseL1DescentAttack(net, eps=12., nb_iter=40, eps_iter=1., l1_sparsity=0.99, rand_init=True)
# attack = CarliniWagnerL2Attack(net, num_classes=10)
net.eval()
evaluation = Evaluation(net)
adv_e = Evaluation(net)

In [7]:
net.eval()


for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs.to(device), targets.to(device)
    # adv_inputs = attack.generate(inputs, targets.reshape(-1,1))
    adv_inputs = attack(inputs)
    evaluation.evaluate(inputs, targets)
    adv_e.evaluate(adv_inputs, targets)
    print(batch_idx)
    print(evaluation.dr)
    print(adv_e.dr)

0
0.98
0.64
1
0.975
0.69
2
0.9733333333333334
0.7
3
0.975
0.725
4
0.974
0.726
5
0.9766666666666667
0.72
6
0.9771428571428571
0.7171428571428572
7
0.97875
0.715
8
0.9811111111111112
0.7233333333333334
9
0.982
0.713
10
0.9827272727272728
0.7072727272727273
11
0.9833333333333333
0.7008333333333333
12
0.9830769230769231
0.7007692307692308
13
0.9828571428571429
0.6978571428571428
14
0.9826666666666667
0.696
15
0.981875
0.69875
16
0.9823529411764705
0.6994117647058824
17
0.9827777777777778
0.6988888888888889
18
0.9821052631578947
0.6963157894736842
19
0.9825
0.6965
20
0.9823809523809524
0.6947619047619048
21
0.9827272727272728
0.6954545454545454
22
0.9826086956521739
0.6947826086956522
23
0.9825
0.6941666666666667
24
0.9824
0.6904
25
0.9826923076923076
0.6915384615384615
26
0.9825925925925926
0.6888888888888889
27
0.9832142857142857
0.6889285714285714
28
0.9837931034482759
0.6879310344827586
29
0.9833333333333333
0.687
30
0.983225806451613
0.6890322580645162
31
0.9828125
0.68875
32
0.9824242